# Lab Assignment 3 - Multiple Linear Regression Solutions
This notebook contains solutions for all questions (Q1, Q2, Q3) in Lab Assignment 3.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

---
# Q1: K-Fold Cross Validation for Multiple Linear Regression (Least Square Error Fit)

**Dataset**: USA House Price Prediction

## Step (a): Load dataset and divide into input features and output variable

In [ ]:
# Load USA Housing dataset
try:
    df_housing = pd.read_csv('USA_Housing.csv')
except:
    # Try alternative path
    df_housing = pd.read_csv('../Assignment_5_Regression/USA_Housing.csv')

print(f"Dataset Shape: {df_housing.shape}")
print(f"\nColumns: {df_housing.columns.tolist()}")
df_housing.head()

In [ ]:
# Separate input features (X) and output variable (y - Price)
# Remove non-numeric columns if any
numeric_cols = df_housing.select_dtypes(include=[np.number]).columns.tolist()

# Assuming 'Price' is the target variable
if 'Price' in numeric_cols:
    X = df_housing[numeric_cols].drop('Price', axis=1)
    y = df_housing['Price']
else:
    # Use last column as target
    X = df_housing[numeric_cols[:-1]]
    y = df_housing[numeric_cols[-1]]

print(f"Input Features Shape: {X.shape}")
print(f"Output Variable Shape: {y.shape}")
print(f"\nFeature names: {X.columns.tolist()}")

## Step (b): Scale the values of input features

In [ ]:
# Scale the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Feature scaling complete!")
print(f"\nBefore scaling - Mean: {X.mean().values[:3]}")
print(f"After scaling - Mean: {X_scaled.mean(axis=0)[:3].round(6)}")
print(f"After scaling - Std: {X_scaled.std(axis=0)[:3].round(6)}")

## Step (c): Divide input and output features into five folds

In [ ]:
# Create 5-fold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

print("5-Fold Cross Validation Setup:")
for fold, (train_idx, test_idx) in enumerate(kf.split(X_scaled), 1):
    print(f"  Fold {fold}: Train size = {len(train_idx)}, Test size = {len(test_idx)}")

## Step (d): Run 5 iterations - Find beta matrix, predicted values, and R2 score using Least Square Error Fit

In [ ]:
def least_square_fit(X, y):
    """
    Calculate beta coefficients using Least Square Error Fit.
    Formula: beta = (X^T * X)^(-1) * X^T * y
    """
    # Add bias term (column of ones)
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    # Calculate beta using normal equation
    beta = np.linalg.pinv(X_b.T @ X_b) @ X_b.T @ y
    return beta

def predict_with_beta(X, beta):
    """Make predictions using beta coefficients."""
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    return X_b @ beta

def calculate_r2(y_true, y_pred):
    """Calculate R-squared score."""
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

print("Least Square Error Fit functions defined!")

In [ ]:
# Run 5-fold cross validation
y_values = y.values if hasattr(y, 'values') else y

fold_results = []

print("="*70)
print("5-FOLD CROSS VALIDATION RESULTS")
print("="*70)

for fold, (train_idx, test_idx) in enumerate(kf.split(X_scaled), 1):
    # Split data
    X_train_fold = X_scaled[train_idx]
    X_test_fold = X_scaled[test_idx]
    y_train_fold = y_values[train_idx]
    y_test_fold = y_values[test_idx]
    
    # Calculate beta using Least Square Error Fit
    beta = least_square_fit(X_train_fold, y_train_fold)
    
    # Make predictions
    y_pred_fold = predict_with_beta(X_test_fold, beta)
    
    # Calculate R2 score
    r2 = calculate_r2(y_test_fold, y_pred_fold)
    
    # Store results
    fold_results.append({
        'fold': fold,
        'beta': beta,
        'r2_score': r2,
        'y_pred': y_pred_fold
    })
    
    print(f"\n--- Fold {fold} ---")
    print(f"Beta (β) matrix shape: {beta.shape}")
    print(f"Beta coefficients (first 3): {beta[:3].round(4)}")
    print(f"R2 Score: {r2:.4f}")

# Summary
r2_scores = [r['r2_score'] for r in fold_results]
print(f"\n{'='*70}")
print(f"SUMMARY")
print(f"{'='*70}")
print(f"Average R2 Score: {np.mean(r2_scores):.4f}")
print(f"R2 Scores: {[round(r, 4) for r in r2_scores]}")

## Step (e): Use best beta matrix for final 70-30 split training and testing

In [ ]:
# Find best beta (highest R2 score)
best_fold = max(fold_results, key=lambda x: x['r2_score'])
best_beta = best_fold['beta']
best_r2 = best_fold['r2_score']

print(f"Best fold: {best_fold['fold']}")
print(f"Best R2 score from CV: {best_r2:.4f}")
print(f"Best Beta coefficients: {best_beta.round(4)}")

In [ ]:
# Final 70-30 split
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_scaled, y_values, test_size=0.3, random_state=42
)

print(f"Final Training set: {X_train_final.shape[0]} samples")
print(f"Final Test set: {X_test_final.shape[0]} samples")

# Train with best beta approach (recalculate on training data)
final_beta = least_square_fit(X_train_final, y_train_final)

# Test performance
y_pred_final = predict_with_beta(X_test_final, final_beta)
final_r2 = calculate_r2(y_test_final, y_pred_final)

print(f"\n{'='*70}")
print("FINAL MODEL PERFORMANCE (70-30 Split)")
print(f"{'='*70}")
print(f"Final R2 Score on Test Set: {final_r2:.4f}")
print(f"Final Beta coefficients: {final_beta.round(4)}")

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: R2 scores across folds
folds = [r['fold'] for r in fold_results]
axes[0].bar(folds, r2_scores, color='steelblue', edgecolor='black')
axes[0].axhline(y=np.mean(r2_scores), color='red', linestyle='--', label=f'Mean R2 = {np.mean(r2_scores):.4f}')
axes[0].set_xlabel('Fold')
axes[0].set_ylabel('R2 Score')
axes[0].set_title('R2 Score Across 5 Folds')
axes[0].legend()

# Plot 2: Actual vs Predicted
axes[1].scatter(y_test_final, y_pred_final, alpha=0.5, color='coral')
axes[1].plot([y_test_final.min(), y_test_final.max()], 
             [y_test_final.min(), y_test_final.max()], 'k--', lw=2)
axes[1].set_xlabel('Actual Price')
axes[1].set_ylabel('Predicted Price')
axes[1].set_title(f'Actual vs Predicted (R2 = {final_r2:.4f})')

plt.tight_layout()
plt.savefig('q1_kfold_results.png', dpi=150, bbox_inches='tight')
plt.show()

---
# Q2: Validation Set for Multiple Linear Regression (Gradient Descent Optimization)

**Dataset**: Same USA Housing dataset

**Split**: Training (56%), Validation (14%), Test (30%)

**Learning rates**: {0.001, 0.01, 0.1, 1}

## Split data into Train (56%), Validation (14%), Test (30%)

In [ ]:
# First split: 70% train+val, 30% test
X_temp, X_test_q2, y_temp, y_test_q2 = train_test_split(
    X_scaled, y_values, test_size=0.3, random_state=42
)

# Second split: 80% train (56% of total), 20% validation (14% of total)
X_train_q2, X_val_q2, y_train_q2, y_val_q2 = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42
)

print(f"Training set: {X_train_q2.shape[0]} samples ({X_train_q2.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"Validation set: {X_val_q2.shape[0]} samples ({X_val_q2.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"Test set: {X_test_q2.shape[0]} samples ({X_test_q2.shape[0]/len(X_scaled)*100:.1f}%)")

## Implement Gradient Descent Optimization

In [ ]:
def gradient_descent_regression(X, y, learning_rate, n_iterations=1000):
    """
    Multiple Linear Regression using Gradient Descent.
    
    Parameters:
    - X: Input features (scaled)
    - y: Target variable
    - learning_rate: Learning rate for gradient descent
    - n_iterations: Number of iterations
    
    Returns:
    - theta: Regression coefficients
    - cost_history: Cost at each iteration
    """
    # Add bias term
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    m, n = X_b.shape
    
    # Initialize theta
    theta = np.zeros(n)
    cost_history = []
    
    for i in range(n_iterations):
        # Predictions
        predictions = X_b @ theta
        
        # Calculate error
        errors = predictions - y
        
        # Calculate gradients
        gradients = (1/m) * (X_b.T @ errors)
        
        # Update theta
        theta = theta - learning_rate * gradients
        
        # Calculate cost (MSE)
        cost = (1/(2*m)) * np.sum(errors**2)
        cost_history.append(cost)
    
    return theta, cost_history

def predict_gd(X, theta):
    """Make predictions using gradient descent coefficients."""
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    return X_b @ theta

print("Gradient Descent functions defined!")

## Train with different learning rates and evaluate

In [ ]:
# Learning rates to test
learning_rates = [0.001, 0.01, 0.1, 1]
n_iterations = 1000

results_q2 = []

print("="*70)
print("GRADIENT DESCENT WITH DIFFERENT LEARNING RATES")
print("="*70)

for lr in learning_rates:
    print(f"\n--- Learning Rate: {lr} ---")
    
    # Train with gradient descent
    theta, cost_history = gradient_descent_regression(X_train_q2, y_train_q2, lr, n_iterations)
    
    # Predictions on validation and test sets
    y_val_pred = predict_gd(X_val_q2, theta)
    y_test_pred = predict_gd(X_test_q2, theta)
    
    # Calculate R2 scores
    val_r2 = calculate_r2(y_val_q2, y_val_pred)
    test_r2 = calculate_r2(y_test_q2, y_test_pred)
    
    results_q2.append({
        'learning_rate': lr,
        'theta': theta,
        'cost_history': cost_history,
        'val_r2': val_r2,
        'test_r2': test_r2,
        'final_cost': cost_history[-1]
    })
    
    print(f"Final Cost: {cost_history[-1]:.4f}")
    print(f"Validation R2 Score: {val_r2:.4f}")
    print(f"Test R2 Score: {test_r2:.4f}")
    print(f"Regression coefficients (first 3): {theta[:3].round(4)}")

In [ ]:
# Find best learning rate based on validation R2
valid_results = [r for r in results_q2 if not np.isnan(r['val_r2']) and not np.isinf(r['val_r2'])]

if valid_results:
    best_result = max(valid_results, key=lambda x: x['val_r2'])
    
    print("\n" + "="*70)
    print("BEST LEARNING RATE SELECTION")
    print("="*70)
    print(f"\nBest Learning Rate: {best_result['learning_rate']}")
    print(f"Validation R2 Score: {best_result['val_r2']:.4f}")
    print(f"Test R2 Score: {best_result['test_r2']:.4f}")
    print(f"Best Regression Coefficients: {best_result['theta'].round(4)}")
else:
    print("No valid results found. Try adjusting learning rates.")

In [ ]:
# Summary table
print("\n" + "="*70)
print("SUMMARY TABLE")
print("="*70)

summary_df = pd.DataFrame([
    {
        'Learning Rate': r['learning_rate'],
        'Final Cost': r['final_cost'],
        'Validation R2': r['val_r2'],
        'Test R2': r['test_r2']
    } for r in results_q2
])
print(summary_df.to_string(index=False))

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Cost convergence for different learning rates
for r in results_q2:
    if r['final_cost'] < 1e10:  # Only plot if converged
        axes[0].plot(r['cost_history'][:200], label=f"LR = {r['learning_rate']}")

axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Cost (MSE)')
axes[0].set_title('Cost Convergence for Different Learning Rates')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: R2 scores comparison
valid_lr = [r['learning_rate'] for r in valid_results]
val_r2s = [r['val_r2'] for r in valid_results]
test_r2s = [r['test_r2'] for r in valid_results]

x = np.arange(len(valid_lr))
width = 0.35

axes[1].bar(x - width/2, val_r2s, width, label='Validation R2', color='steelblue')
axes[1].bar(x + width/2, test_r2s, width, label='Test R2', color='coral')
axes[1].set_xlabel('Learning Rate')
axes[1].set_ylabel('R2 Score')
axes[1].set_title('R2 Scores for Different Learning Rates')
axes[1].set_xticks(x)
axes[1].set_xticklabels(valid_lr)
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('q2_gradient_descent_results.png', dpi=150, bbox_inches='tight')
plt.show()

---
# Q3: Pre-processing and Multiple Linear Regression with PCA

**Dataset**: Car Price Prediction (imports-85.data)

## Step 1: Load dataset with column names and replace '?' with NaN

In [ ]:
# Column names as specified
column_names = [
    "symboling", "normalized_losses", "make", "fuel_type", "aspiration",
    "num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base",
    "length", "width", "height", "curb_weight", "engine_type", "num_cylinders",
    "engine_size", "fuel_system", "bore", "stroke", "compression_ratio",
    "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"
]

# Load dataset
try:
    df_cars = pd.read_csv('imports-85.csv', header=None, names=column_names, na_values='?')
except:
    # Try URL
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
    df_cars = pd.read_csv(url, header=None, names=column_names, na_values='?')

print(f"Dataset Shape: {df_cars.shape}")
print(f"\nMissing values:\n{df_cars.isnull().sum()[df_cars.isnull().sum() > 0]}")
df_cars.head()

## Step 2: Replace NaN with central tendency imputation, drop rows with NaN in price

In [ ]:
# Drop rows with NaN in price column
df_cars = df_cars.dropna(subset=['price'])
print(f"After dropping NaN in price: {df_cars.shape}")

# Fill remaining NaN with central tendency (median for numeric, mode for categorical)
for col in df_cars.columns:
    if df_cars[col].dtype == 'object':
        df_cars[col].fillna(df_cars[col].mode()[0], inplace=True)
    else:
        df_cars[col].fillna(df_cars[col].median(), inplace=True)

print(f"After imputation - Missing values: {df_cars.isnull().sum().sum()}")

## Step 3: Convert non-numeric values to numeric

In [ ]:
# (i) num_doors and num_cylinders: convert words to numbers
word_to_num = {
    'two': 2, 'three': 3, 'four': 4, 'five': 5, 
    'six': 6, 'eight': 8, 'twelve': 12
}

df_cars['num_doors'] = df_cars['num_doors'].replace(word_to_num)
df_cars['num_cylinders'] = df_cars['num_cylinders'].replace(word_to_num)
print("(i) num_doors and num_cylinders converted to numbers")

# (ii) body_style, drive_wheels: dummy encoding
df_cars = pd.get_dummies(df_cars, columns=['body_style', 'drive_wheels'], drop_first=True)
print("(ii) body_style and drive_wheels: dummy encoding applied")

# (iii) make, aspiration, engine_location, fuel_type: label encoding
label_cols = ['make', 'aspiration', 'engine_location', 'fuel_type']
le = LabelEncoder()
for col in label_cols:
    df_cars[col] = le.fit_transform(df_cars[col].astype(str))
print("(iii) make, aspiration, engine_location, fuel_type: label encoding applied")

# (iv) fuel_system: replace 'pfi' containing values to 1, else 0
df_cars['fuel_system'] = df_cars['fuel_system'].apply(
    lambda x: 1 if 'pfi' in str(x).lower() else 0
)
print("(iv) fuel_system: pfi -> 1, else -> 0")

# (v) engine_type: replace 'ohc' containing values to 1, else 0
df_cars['engine_type'] = df_cars['engine_type'].apply(
    lambda x: 1 if 'ohc' in str(x).lower() else 0
)
print("(v) engine_type: ohc -> 1, else -> 0")

print(f"\nDataset shape after encoding: {df_cars.shape}")

## Step 4: Divide into input/output and scale features

In [ ]:
# Separate features and target
X_cars = df_cars.drop('price', axis=1)
y_cars = df_cars['price']

# Convert all to numeric
X_cars = X_cars.apply(pd.to_numeric, errors='coerce')
X_cars = X_cars.fillna(0)

# Scale features
scaler_cars = StandardScaler()
X_cars_scaled = scaler_cars.fit_transform(X_cars)

print(f"Features shape: {X_cars_scaled.shape}")
print(f"Target shape: {y_cars.shape}")

## Step 5: Train linear regressor on 70% data, test on 30%

In [ ]:
# Train-test split
X_train_cars, X_test_cars, y_train_cars, y_test_cars = train_test_split(
    X_cars_scaled, y_cars, test_size=0.3, random_state=42
)

print(f"Training set: {X_train_cars.shape[0]} samples")
print(f"Test set: {X_test_cars.shape[0]} samples")

# Train Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_cars, y_train_cars)

# Predictions
y_pred_cars = lr_model.predict(X_test_cars)

# Evaluate
r2_no_pca = r2_score(y_test_cars, y_pred_cars)
rmse_no_pca = np.sqrt(mean_squared_error(y_test_cars, y_pred_cars))

print(f"\n{'='*60}")
print("LINEAR REGRESSION WITHOUT PCA")
print(f"{'='*60}")
print(f"R2 Score: {r2_no_pca:.4f}")
print(f"RMSE: {rmse_no_pca:.2f}")
print(f"Number of features: {X_train_cars.shape[1]}")

## Step 6: Apply PCA and retrain linear regressor

In [ ]:
# Apply PCA (retain 95% variance)
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_cars)
X_test_pca = pca.transform(X_test_cars)

print(f"Original features: {X_train_cars.shape[1]}")
print(f"Features after PCA: {X_train_pca.shape[1]}")
print(f"Variance explained: {pca.explained_variance_ratio_.sum()*100:.2f}%")

In [ ]:
# Train Linear Regression on PCA-reduced data
lr_model_pca = LinearRegression()
lr_model_pca.fit(X_train_pca, y_train_cars)

# Predictions
y_pred_pca = lr_model_pca.predict(X_test_pca)

# Evaluate
r2_with_pca = r2_score(y_test_cars, y_pred_pca)
rmse_with_pca = np.sqrt(mean_squared_error(y_test_cars, y_pred_pca))

print(f"\n{'='*60}")
print("LINEAR REGRESSION WITH PCA")
print(f"{'='*60}")
print(f"R2 Score: {r2_with_pca:.4f}")
print(f"RMSE: {rmse_with_pca:.2f}")
print(f"Number of features: {X_train_pca.shape[1]}")

In [ ]:
# Comparison
print(f"\n{'='*60}")
print("COMPARISON: WITH vs WITHOUT PCA")
print(f"{'='*60}")

comparison_pca = pd.DataFrame({
    'Model': ['Without PCA', 'With PCA'],
    'Features': [X_train_cars.shape[1], X_train_pca.shape[1]],
    'R2 Score': [r2_no_pca, r2_with_pca],
    'RMSE': [rmse_no_pca, rmse_with_pca]
})
print(comparison_pca.to_string(index=False))

if r2_with_pca > r2_no_pca:
    print(f"\nConclusion: Yes, PCA led to performance improvement! (+{(r2_with_pca-r2_no_pca)*100:.2f}%)")
else:
    print(f"\nConclusion: PCA did not improve R2, but reduced features from {X_train_cars.shape[1]} to {X_train_pca.shape[1]}")

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Plot 1: R2 Comparison
models = ['Without PCA', 'With PCA']
r2_scores = [r2_no_pca, r2_with_pca]
colors = ['steelblue', 'coral']
axes[0].bar(models, r2_scores, color=colors)
axes[0].set_ylabel('R2 Score')
axes[0].set_title('R2 Score Comparison')
for i, v in enumerate(r2_scores):
    axes[0].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold')

# Plot 2: Explained Variance by PCA components
cum_var = np.cumsum(pca.explained_variance_ratio_)
axes[1].plot(range(1, len(cum_var)+1), cum_var, 'bo-')
axes[1].axhline(y=0.95, color='r', linestyle='--', label='95% threshold')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].set_title('PCA Explained Variance')
axes[1].legend()

# Plot 3: Actual vs Predicted (With PCA)
axes[2].scatter(y_test_cars, y_pred_pca, alpha=0.5, color='coral')
axes[2].plot([y_test_cars.min(), y_test_cars.max()], 
             [y_test_cars.min(), y_test_cars.max()], 'k--', lw=2)
axes[2].set_xlabel('Actual Price')
axes[2].set_ylabel('Predicted Price')
axes[2].set_title(f'Actual vs Predicted (PCA, R2={r2_with_pca:.4f})')

plt.tight_layout()
plt.savefig('q3_pca_results.png', dpi=150, bbox_inches='tight')
plt.show()

---
# Summary

- **Q1**: Implemented 5-Fold Cross Validation for Multiple Linear Regression using Least Square Error Fit
- **Q2**: Used Gradient Descent Optimization with different learning rates and validation set approach
- **Q3**: Applied data preprocessing, encoding, and PCA for dimensionality reduction on car price prediction